<a href="https://colab.research.google.com/github/Rafi076/RTFER/blob/main/Copy_of_ER2013_Ensemble_EffNetB3_ResNet50_Xception.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import kagglehub
path = kagglehub.dataset_download("msambare/fer2013")

Using Colab cache for faster access to the 'fer2013' dataset.


In [22]:
import os

print("Dataset root path:")
print(path)

print("\nContents of dataset root:")
print(os.listdir(path))


Dataset root path:
/kaggle/input/fer2013

Contents of dataset root:
['test', 'train']


**1. Environment & Library Setup**

In [23]:
# Core deep learning
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, optimizers, callbacks

# Numerical & data handling
import numpy as np
import pandas as pd

# Image processing
import cv2
from skimage import exposure, filters

# Machine learning utilities
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    classification_report,
    roc_auc_score
)

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# System & utility
import os
import random
import warnings
warnings.filterwarnings("ignore")

# Environment check
print("TensorFlow version:", tf.__version__)
print("GPU Available:", tf.config.list_physical_devices('GPU'))


TensorFlow version: 2.19.0
GPU Available: []


------

**2. Preprocessing Pipeline**

In [24]:
# Imports for preprocessing
from tensorflow.keras.applications import (
    resnet50,
    xception,
    efficientnet
)


2.1: *Face-Aware Center Alignment (lightweight & safe)*

FER-2013 faces are mostly centered, so we use center cropping + landmark-free alignment
(this avoids unstable landmark detectors on low-res faces).

In [25]:
def face_aware_center_align(img, crop_ratio=0.9):
    """
    Light face-aware center alignment using center crop.
    Safe for FER-2013.
    """
    h, w = img.shape[:2]
    crop_h, crop_w = int(h * crop_ratio), int(w * crop_ratio)

    start_x = (w - crop_w) // 2
    start_y = (h - crop_h) // 2

    return img[start_y:start_y + crop_h, start_x:start_x + crop_w]


2.2: *CLAHE (Contrast Enhancement)*

In [26]:
def apply_clahe(gray_img):
    clahe = cv2.createCLAHE(
        clipLimit=2.0,
        tileGridSize=(8, 8)
    )
    return clahe.apply(gray_img)


2.2.1 ***Gamma Correction*** (This balances the lighting before you enhance contrast)  &    ***NLM Denoising*** (Upgrade from Gaussian. Gaussian blur can "muddy" the edges of the lips. Non-Local Means (NLM) denoising is superior because it removes grain while keeping edges razor-sharp)

In [27]:
def apply_gamma_correction(img, gamma=1.1):
    """Normalizes global illumination before contrast enhancement."""
    invGamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** invGamma) * 255
                      for i in np.arange(0, 256)]).astype("uint8")
    return cv2.LUT(img, table)

def apply_denoising(img):
    """Removes digital grain while preserving sharp edges of lips/eyes."""
    # fastNlMeans is superior to Gaussian for keeping micro-expression edges
    return cv2.fastNlMeansDenoising(img, None, h=3, templateWindowSize=7, searchWindowSize=21)

2.3: *Light Gaussian Smoothing*

In [28]:
def gaussian_smoothing(img):
    return cv2.GaussianBlur(img, (3, 3), 0)


2.4: *Grayscale → RGB Conversion*

In [29]:
def gray_to_rgb(gray_img):
    return cv2.cvtColor(gray_img, cv2.COLOR_GRAY2RGB)


2.5: *Model-Specific Resize*

In [30]:
def resize_for_model(img, model_name):
    if model_name == "resnet50":
        return cv2.resize(img, (224, 224))
    elif model_name == "xception":
        return cv2.resize(img, (299, 299))
    elif model_name == "efficientnetb3":
        return cv2.resize(img, (300, 300))
    else:
        raise ValueError("Unknown model name")


2.6: *Model-Specific Normalization*

In [31]:
def preprocess_for_model(img, model_name):
    img = img.astype(np.float32)

    if model_name == "resnet50":
        return resnet50.preprocess_input(img)
    elif model_name == "xception":
        return xception.preprocess_input(img)
    elif model_name == "efficientnetb3":
        return efficientnet.preprocess_input(img)
    else:
        raise ValueError("Unknown model name")


2.7: *FULL Preprocessing Function*

In [33]:
def preprocess_image(image_path, model_name):
    # 1. Read image (FER-2013 grayscale)
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # 2. Face-aware alignment (Your crop)
    img = face_aware_center_align(img)

    # 3. [NEW] Gamma Correction (Balance lighting FIRST)
    img = apply_gamma_correction(img, gamma=1.2)

    # 4. [NEW] Denoising (Clean grain BEFORE sharpening)
    img = apply_denoising(img)

    # 5. CLAHE (Contrast Enhancement - now works on clean pixels)
    img = apply_clahe(img)

    # 6. Gray → RGB (Convert for ImageNet weights)
    img = gray_to_rgb(img)

    # 7. Resize (Model-specific)
    target_size = (224, 224)
    if model_name == "xception": target_size = (299, 299)
    if model_name == "efficientnetb3": target_size = (300, 300)

    img = cv2.resize(img, target_size, interpolation=cv2.INTER_CUBIC)

    # 8. Model-specific normalization
    img = preprocess_for_model(img, model_name)

    return img

------------------------


3. **Dataset Splitting**

3.1: *Define dataset paths*

In [40]:
DATASET_ROOT = path  # /kaggle/input/fer2013
TRAIN_DIR = os.path.join(DATASET_ROOT, "train")
TEST_DIR  = os.path.join(DATASET_ROOT, "test")

print("Train dir:", TRAIN_DIR)
print("Test dir :", TEST_DIR)


Train dir: /kaggle/input/fer2013/train
Test dir : /kaggle/input/fer2013/test


3.2: *Read all training image paths & labels*

In [41]:
def load_image_paths_and_labels(root_dir):
    image_paths = []
    labels = []

    for label_name in sorted(os.listdir(root_dir)):
        label_path = os.path.join(root_dir, label_name)
        if not os.path.isdir(label_path):
            continue
        for img_name in os.listdir(label_path):
            image_paths.append(os.path.join(label_path, img_name))
            labels.append(label_name)
    return image_paths, labels

train_image_paths, train_labels = load_image_paths_and_labels(TRAIN_DIR)
test_image_paths, test_labels = load_image_paths_and_labels(TEST_DIR)

print("Total training images:", len(train_image_paths))
print("Total test images    :", len(test_image_paths))


Total training images: 28709
Total test images    : 7178


3.3: *Encode labels (string → integer*

In [42]:
# Hard-coding classes ensures indices NEVER change between splits
classes = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
label_encoder = LabelEncoder()
label_encoder.fit(classes)  # Fit on known labels, NOT folder names

train_labels_encoded = label_encoder.transform(train_labels)
test_labels_encoded  = label_encoder.transform(test_labels)

NUM_CLASSES = len(label_encoder.classes_)
print("Classes:", label_encoder.classes_)
print("Number of classes:", NUM_CLASSES)

Classes: ['angry' 'disgust' 'fear' 'happy' 'neutral' 'sad' 'surprise']
Number of classes: 7


3.4: Split Train → Train (70%) + Validation (15%)

We do:

85% → temp (train+val)

15% → validation

Then split temp into 70% train *italicized text*

In [43]:
# Step 1: Split off 15% for Validation
X_temp, X_val, y_temp, y_val = train_test_split(
    train_image_paths,
    train_labels_encoded,
    test_size=0.15,
    stratify=train_labels_encoded,
    random_state=42
)

# Step 2: Further split X_temp to get ~70% final training
X_train, X_unused, y_train, y_unused = train_test_split(
    X_temp,
    y_temp,
    test_size=0.1765,  # makes final train ≈70%
    stratify=y_temp,
    random_state=42
)

print("Train size     :", len(X_train))
print("Validation size:", len(X_val))
print("Test size      :", len(test_image_paths))


Train size     : 20095
Validation size: 4307
Test size      : 7178


3.5: *Load TEST set*

In [44]:
unique, counts = np.unique(y_train, return_counts=True)
print("Class distribution in Final Train:", dict(zip(label_encoder.classes_, counts)))

Class distribution in Final Train: {np.str_('angry'): np.int64(2797), np.str_('disgust'): np.int64(306), np.str_('fear'): np.int64(2867), np.str_('happy'): np.int64(5051), np.str_('neutral'): np.int64(3475), np.str_('sad'): np.int64(3380), np.str_('surprise'): np.int64(2219)}


------